In [2]:
#pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 53.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.0/512.0 kB 22.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [5]:
#pip install selenium webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [8]:
#pip install yt-dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 3.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 42.4 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [18]:
import yt_dlp
import json
from datetime import datetime
import os

class YouTubeShortsScaper:
    def __init__(self):
        self.ydl_opts = {
            'quiet': True,
            'no_warnings': True,
            'extract_flat': False,
            'skip_download': True,
        }
    
    def scrape_short(self, url):
        print(f"\n{'='*60}")
        print(f" Extraction en cours: {url}")
        print('='*60)
        
        try:
            with yt_dlp.YoutubeDL(self.ydl_opts) as ydl:
                info = ydl.extract_info(url, download=False)
                
                # Extraire les données pertinentes
                data = {
                    'url': url,
                    'video_id': info.get('id'),
                    'titre': info.get('title'),
                    'description': info.get('description', '')[:200] + '...' if info.get('description') else None,
                    'chaine': info.get('uploader') or info.get('channel'),
                    'chaine_id': info.get('channel_id'),
                    'date_publication': info.get('upload_date'),
                    'duree_secondes': info.get('duration'),
                    'nombre_vues': info.get('view_count'),
                    'nombre_likes': info.get('like_count'),
                    'nombre_commentaires': info.get('comment_count'),
                    'miniature': info.get('thumbnail'),
                    #'tags': info.get('tags', [])[:5],
                }
                
                # Formater la date
                if data['date_publication']:
                    try:
                        date_obj = datetime.strptime(data['date_publication'], '%Y%m%d')
                        data['date_formatee'] = date_obj.strftime('%d/%m/%Y')
                    except:
                        data['date_formatee'] = data['date_publication']
                
            
                
                # Afficher le résumé
                print(f" Titre: {data['titre']}")
                print(f" Chaîne: {data['chaine']}")
                print(f" Date: {data.get('date_formatee', 'N/A')}")
                print(f" Vues: {self.format_number(data['nombre_vues'])}")
                print(f" Likes: {self.format_number(data['nombre_likes'])}")
                print(f" Commentaires: {self.format_number(data['nombre_commentaires'])}")
                print(f" Durée: {data['duree_secondes']}s")
                #print(f" {len(data['commentaires'])} commentaires extraits")
                
                return data
                
        except Exception as e:
            print(f" Erreur lors de l'extraction de {url}: {str(e)}")
            return None
    
    def format_number(self, num):
        if num is None:
            return "N/A"
        if num >= 1_000_000:
            return f"{num/1_000_000:.1f}M"
        elif num >= 1_000:
            return f"{num/1_000:.1f}K"
        return str(num)
    
    def scrape_multiple_shorts(self, urls):
        results = []
        print(f"\n Début de l'extraction de {len(urls)} YouTube Short(s)...")
        
        for i, url in enumerate(urls, 1):
            print(f"\n[{i}/{len(urls)}]")
            data = self.scrape_short(url)
            if data:
                results.append(data)
        
        return results
    
    def save_to_json(self, data, filename=None):
        if filename is None:
            # Créer un nom de fichier avec timestamp
            timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
            filename = f'shorts_data_{timestamp}.json'
        
        # S'assurer que le fichier a l'extension .json
        if not filename.endswith('.json'):
            filename += '.json'
        
        try:
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(data, f, indent=2, ensure_ascii=False)
            
            # Obtenir le chemin absolu
            chemin_absolu = os.path.abspath(filename)
            print(f"\n Données sauvegardées avec succès!")
            print(f" Fichier: {filename}")
            print(f" Chemin complet: {chemin_absolu}")
            print(f" Taille: {os.path.getsize(filename)} octets")
            
            return chemin_absolu
        except Exception as e:
            print(f" Erreur lors de la sauvegarde: {e}")
            return None
    
    
def get_user_input():
    """Demander les URLs à l'utilisateur"""
    print("\n" + "="*60)
    print(" SCRAPER DE YOUTUBE SHORTS")
    print("="*60)
    print("\nEntrez les liens des YouTube Shorts à extraire.")
    print("Vous pouvez entrer plusieurs liens (un par ligne).")
    print("Tapez 'fin' ou laissez vide pour terminer.\n")
    
    urls = []
    while True:
        url = input(f"Short #{len(urls) + 1} (ou 'fin' pour terminer): ").strip()
        
        if url.lower() == 'fin' or url == '':
            break
        
        # Vérifier si c'est un lien YouTube valide
        if 'youtube.com/shorts/' in url or 'youtu.be/' in url:
            urls.append(url)
            print(f" Lien ajouté: {url}\n")
        else:
            print("  Ce ne semble pas être un lien YouTube Shorts valide.")
            print("   Format attendu: https://www.youtube.com/shorts/xxxxx\n")
    
    return urls


def get_filename():
    print("\n" + "-"*60)
    default_name = f"shorts_data_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    filename = input(f"Nom du fichier JSON (appuyez sur Entrée pour '{default_name}'): ").strip()
    
    if not filename:
        filename = default_name
    
    return filename


# Programme principal
if __name__ == "__main__":
    try:
        # Demander les URLs à l'utilisateur
        shorts_urls = get_user_input()
        
        if not shorts_urls:
            print("\n  Aucun lien fourni. Au revoir!")
            exit()
        
        print(f"\n {len(shorts_urls)} lien(s) à extraire.")
        
        # Demander le nom du fichier
        filename = get_filename()
        
        # Initialiser le scraper
        scraper = YouTubeShortsScaper()
        
        # Extraire les données
        results = scraper.scrape_multiple_shorts(shorts_urls)
        
        if results:
            # Afficher le résumé
            
            
            # Sauvegarder en JSON
            saved_path = scraper.save_to_json(results, filename)
            
            if saved_path:
                print("\n" + "="*60)
                print(" EXTRACTION TERMINÉE AVEC SUCCÈS!")
                print("="*60)
                print(f"\n Vous pouvez maintenant ouvrir le fichier:")
                print(f"   {saved_path}")
        else:
            print("\n Aucune donnée extraite. Vérifiez vos liens.")
        
    except KeyboardInterrupt:
        print("\n\n  Programme interrompu par l'utilisateur.")


 SCRAPER DE YOUTUBE SHORTS

Entrez les liens des YouTube Shorts à extraire.
Vous pouvez entrer plusieurs liens (un par ligne).
Tapez 'fin' ou laissez vide pour terminer.



Short #1 (ou 'fin' pour terminer):  https://www.youtube.com/shorts/RXMlrWb0NWc


 Lien ajouté: https://www.youtube.com/shorts/RXMlrWb0NWc



Short #2 (ou 'fin' pour terminer):  https://www.youtube.com/shorts/_HOnnFT3A5o


 Lien ajouté: https://www.youtube.com/shorts/_HOnnFT3A5o



...

 Lien ajouté: https://www.youtube.com/shorts/04Ptt0IXsMM



Short #5 (ou 'fin' pour terminer):  fin



 4 lien(s) à extraire.

------------------------------------------------------------


Nom du fichier JSON (appuyez sur Entrée pour 'shorts_data_20251211_181606.json'):  test_scrap



 Début de l'extraction de 4 YouTube Short(s)...

[1/4]

 Extraction en cours: https://www.youtube.com/shorts/RXMlrWb0NWc
 Titre: يعطيك الصحة فطّومة، ما تتصوّرش قدّاش قاعدة تكبر في عيني🤣#شوفلي_حل
 Chaîne: Choufli Hal - شوفلي حل
 Date: 09/12/2025
 Vues: 23.0K
 Likes: 340
 Commentaires: 4
 Durée: 30s

[2/4]

 Extraction en cours: https://www.youtube.com/shorts/_HOnnFT3A5o
 Titre: باب الحارة  ـ  أم ذكي عم تخبر جميلة بأنو رح يجيهااا ضرة
 Chaîne: باب الحارة Bab Alhara
 Date: 10/12/2025
 Vues: 52.7K
 Likes: 1.7K
 Commentaires: 13
 Durée: 109s

[3/4]

 Extraction en cours: https://www.youtube.com/shorts/VFusGeJ-VLY
 Titre: كواليس نسيبتي العزيزة ج2
 Chaîne: Dream 21
 Date: 15/10/2024
 Vues: 878.1K
 Likes: 19.9K
 Commentaires: 97
 Durée: 59s

[4/4]

 Extraction en cours: https://www.youtube.com/shorts/04Ptt0IXsMM
 Titre: اسرار ال drop shipping  في تونس #shorts
 Chaîne: OLWEN AWANI - L'Expert Tunisien
 Date: 10/03/2022
 Vues: 21.8K
 Likes: 751
 Commentaires: 39
 Durée: 59s

 Données sauvegardé